In [1]:
%load_ext snakeviz
%config InlineBackend.figure_format = 'svg'
%config InlineBackend.figure_format = 'png'

In [37]:
from email import generator
import numpy as np
from python.ADMPForce import ADMPGenerator
import scipy
from scipy.stats import special_ortho_group
from simtk.openmm import *
from simtk.openmm.app import *
from simtk.unit import *
import mpidplugin

mScales = np.array([0.0, 0.0, 0.0, 1.0])
pScales = np.array([0.0, 0.0, 0.0, 1.0])
dScales = np.array([0.0, 0.0, 0.0, 1.0])
rc = 8 # in Angstrom
ethresh = 1e-4


pdb = 'tests/samples/waterdimer_aligned.pdb'
xml = 'tests/samples/mpidwater.xml'
generator = ADMPGenerator(pdb, xml, rc, ethresh, mScales, pScales, dScales, )
# get a random geometry for testing
scipy.random.seed(1000)
R1 = special_ortho_group.rvs(3)
R2 = special_ortho_group.rvs(3)

positions = generator.positions
positions[0:3] = positions[0:3].dot(R1)
positions[3:6] = positions[3:6].dot(R2)
positions[3:] += np.array([3.0, 0.0, 0.0])


force = generator.create_force()
force.update()
force.kappa = 0.328532611


In [16]:
print(positions)

[[ 0.          0.          0.        ]
 [-0.72004306 -0.01869945  0.63058173]
 [ 0.68557522  0.50825211  0.43371812]
 [ 3.          0.          0.        ]
 [ 3.02161682 -0.52203431 -0.80215827]
 [ 2.08174622 -0.01347945  0.27032073]]


In [40]:
ene0 = force.calc_reci_space_energy()

In [41]:
print(ene0)

3.850401016246608


In [42]:
forces = force.calc_reci_space_force()

In [43]:
print(forces)

[[  5.58156365  -2.58004665 -10.82644139]
 [ -2.49424205   1.30562368   5.67417203]
 [ -3.48460866   1.52578869   5.09100699]
 [  5.03836956   1.55864688  -0.65421062]
 [ -0.98473687  -2.01505676  -1.87536137]
 [ -3.65628997   0.20508466   2.59076499]]


In [38]:
N = np.array([force.K1, force.K2, force.K3])
Q = force.Q[:, :(force.lmax+1)**2].reshape(force.Q.shape[0], (force.lmax+1)**2)
from python.pme import pme_reciprocal
from jax.test_util import check_grads
ene0 = pme_reciprocal(force.positions, force.box, Q, force.lmax, force.kappa, N)

In [39]:
print(ene0)

3.850401016246608


In [46]:
delta = np.zeros((6,3))
delta[2,2] = 1e-5

print((pme_reciprocal(force.positions+delta, force.box, Q, force.lmax, force.kappa, N) - ene0)/1e-5)